In [1]:
function mcmcpi(L=10^9)
    c = 0
    for i in 1:L
        c += rand()^2 + rand()^2 ≤ 1
    end
    4c/L
end

@time mcmcpi()
@time mcmcpi()
@time mcmcpi()

  3.004806 seconds
  3.017117 seconds
  3.014180 seconds


3.141549932

In [2]:
function mcmcpi_threads(L=10^9)
    c = zeros(Int, Threads.nthreads())
    Threads.@threads for i in 1:L
        tid = Threads.threadid()
        c[tid] += rand()^2 + rand()^2 ≤ 1
    end
    4sum(c)/L
end

@time mcmcpi_threads()
@time mcmcpi_threads()
@time mcmcpi_threads()

  0.584347 seconds (14.40 k allocations: 963.016 KiB, 53.77% compilation time)
  0.561709 seconds (186 allocations: 18.641 KiB)
  0.574157 seconds (185 allocations: 18.266 KiB)


3.141582072

In [3]:
using LoopVectorization

# The result shall be 4.0 or 0.0.
function mcmcpi_turbo_incorrect(L=10^9)
    c = 0
    @turbo for i in 1:L
        c += rand()^2 + rand()^2 ≤ 1
    end
    4c/L
end

@time mcmcpi_turbo_incorrect()
@time mcmcpi_turbo_incorrect()
@time mcmcpi_turbo_incorrect()

  0.010849 seconds
  0.009870 seconds
  0.010287 seconds


4.0

In [4]:
using LoopVectorization

rand_is_in_unit_circle(i) = rand()^2 + rand()^2 ≤ 1

function mcmcpi_turbo(L=10^9)
    c = 0
    @turbo for i in 1:L
        c += rand_is_in_unit_circle(i)
    end
    4c/L
end

@time mcmcpi_turbo()
@time mcmcpi_turbo()
@time mcmcpi_turbo()

  0.490229 seconds
  0.496497 seconds
  0.490500 seconds


3.141767424